In [1]:
import os
import json

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.utils import Sequence, to_categorical

print(tf.test.is_gpu_available())

max_frame = 60

/home/jules/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jules/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jules/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jules/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

True


https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

https://pytorch.org/docs/stable/data.html

https://github.com/kenshohara/3D-ResNets-PyTorch

In [2]:
list_IDs = {}
with open("data/dataset.json", "r") as file:
    list_IDs = json.load(file)
labels = {}
with open("data/labels.json", "r") as file:
    labels = json.load(file)

In [3]:
class NTUSequence(Sequence):
    
    def __init__(self, list_IDs, labels, path, batch_size=16, max_frame=max_frame, one_hot=True):
        self.x = list_IDs
        self.y = labels
        self.batch_size = batch_size
        self.max_frame = max_frame
        self.path = path
        if one_hot:
            self.one_hot_encode()

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = np.array([self.y[ID] for ID in batch_x])
        
        X = [np.load(self.path + ID + '.npy')[:, :, :, np.newaxis] for ID in batch_x]
        
        if not(self.max_frame is None):
            X = np.stack(
                [x[:max_frame] if x.shape[0] >= max_frame else np.pad(x, ((0, max_frame-x.shape[0]),(0,0),(0,0),(0,0)), "constant") for x in X]
            )
        
        return X, batch_y

    def one_hot_encode(self):
        labels = [val for val in self.y.values()]
        ids = [key for key in self.y.keys()]
        labels = to_categorical(labels)
        self.y = {ID: label for ID, label in zip(ids, labels)}

In [4]:
trainset = NTUSequence(list_IDs["train"], labels, path="data/processed/train/")
testset = NTUSequence(list_IDs["validation"], labels, path="data/processed/test/")

# Conv3D 

In [16]:
conv3d = models.Sequential([
    layers.Conv3D(64, 3, activation="relu", padding="same", input_shape=(max_frame, 25, 25, 1)),
    layers.Conv3D(64, 3, activation="relu", padding="same"),
    layers.MaxPooling3D((2,2,2)),
    
    layers.Conv3D(128, 3, activation="relu", padding="same"),
    layers.Conv3D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling3D((2,2,2)),
    
    layers.Conv3D(256, 3, activation="relu", padding="same"),
    layers.Conv3D(256, 3, activation="relu", padding="same"),
    layers.MaxPooling3D((2,2,2)),
    
    layers.Flatten(),
    #layers.Dense(1, activation="relu"),
    layers.Dense(49, activation="softmax")
])

conv3d.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_6 (Conv3D)            (None, 40, 25, 25, 64)    1792      
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 40, 25, 25, 64)    110656    
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 20, 12, 12, 64)    0         
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 20, 12, 12, 128)   221312    
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 20, 12, 12, 128)   442496    
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 10, 6, 6, 128)     0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 10, 6, 6, 256)    

In [17]:
conv3d.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
import datetime 

log_dir="logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
print(log_dir)

history = conv3d.fit_generator(trainset, 
                     epochs=20,
                   callbacks=[tensorboard_callback])

# LSTM

In [5]:
convnet = models.Sequential([
    
    layers.Conv2D(64, 3, activation="relu", padding="same", input_shape=(25, 25, 1)),
    layers.Conv2D(64, 3, activation="relu", padding="same"),
    layers.Conv2D(64, 3, activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten()
                        ])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
convnet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 25, 25, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 64)        36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 128)         0

In [7]:

LSTM = models.Sequential([
    layers.Input((None, 25, 25, 1)),
    layers.TimeDistributed(convnet),
    
    layers.LSTM(80, activation="relu"),
    
    layers.Dense(49, activation="softmax")
])

In [8]:


adam = tf.keras.optimizers.Adam(lr=1e-3, decay=1e-8)
LSTM.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
LSTM.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, None, 2304)        1181184   
_________________________________________________________________
lstm (LSTM)                  (None, 80)                763200    
_________________________________________________________________
dense (Dense)                (None, 49)                3969      
Total params: 1,948,353
Trainable params: 1,948,353
Non-trainable params: 0
_________________________________________________________________


In [10]:
import datetime 

log_dir="logs/LSTM/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
print(log_dir)

history = LSTM.fit_generator(trainset, 
                     epochs=20,
                   callbacks=[tensorboard_callback])

logs/LSTM/fit/20200130-192105
Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[16] and type int64 on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node metrics/acc/ArgMax}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[loss/mul/_151]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[16] and type int64 on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node metrics/acc/ArgMax}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 derived errors ignored.